<a href="https://colab.research.google.com/github/Shreyasii11/Shreyasii11/blob/main/HOG_with_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Define the size of the sliding window and the stride
WINDOW_SIZE = (64, 64)
STRIDE = 8


In [ ]:
# Load the image dataset
data = []
labels = []
for folder_name in ['Oranges', 'Non-Oranges']:
    folder_path = f'/content/drive/My Drive/Fruit_Dataset/{folder_name}/'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_gray, WINDOW_SIZE)
        HOG_PARAMS = {'_winSize': (64,64),
              '_blockSize': (16, 16),
              '_blockStride': (8, 8),
              '_cellSize': (8, 8),
              '_nbins': 9,
              '_derivAperture': 1,
              '_winSigma': -1,
              '_histogramNormType': 0,
              '_L2HysThreshold': 0.2,
              '_gammaCorrection': True,
              '_nlevels': 64,
              '_signedGradient': False}

        hog = cv2.HOGDescriptor(**HOG_PARAMS)
        features = hog.compute(img_resized)
        data.append(features)
        labels.append(1 if folder_name == 'Oranges' else 0)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
# Train an SVM classifier on the HOG features
svm = SVC(kernel='linear', C=100,gamma='auto')
svm.fit(X_train, y_train)

SVC(C=100, gamma='auto', kernel='linear')

In [ ]:
# Evaluate the accuracy of the classifier on the testing set
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print(f'Accuracy: {accuracy}')

Accuracy: 75.0


In [ ]:
# Count the number of oranges in a new image
img_path = '/content/test_image.jpg'
img = cv2.imread(img_path)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
num_oranges = 0
for y in range(0, img_gray.shape[0] - WINDOW_SIZE[0], STRIDE):
    for x in range(0, img_gray.shape[1] - WINDOW_SIZE[1], STRIDE):
        window = img_gray[y:y+WINDOW_SIZE[0], x:x+WINDOW_SIZE[1]]
        window_resized = cv2.resize(window, WINDOW_SIZE)
        hog = cv2.HOGDescriptor(**HOG_PARAMS)
        features = hog.compute(window_resized).reshape(1, -1)
        pred = svm.predict(features)[0]
        if pred == 1:
            num_oranges += 1

In [ ]:
print(f'Number of oranges: {num_oranges}')

Number of oranges: 180
